In [75]:
import pandas as pd
import numpy as np

In [76]:
df = pd.read_csv('../data/googleplaystore.csv')
df['Genres'] = df.Genres.str.split(';')

In [77]:
genres = ["Action","Action & Adventure","Adventure","Art & Design","Arcade","Auto & Vehicles","Beauty","Board","Books & Reference","Brain Games","Business","Card","Casino","Casual","Comics","Communication","Creativity","Dating","Education","Educational","Entertainment","Events","Finance","Food & Drink","Health & Fitness","House & Home","Libraries & Demo","Lifestyle","Maps & Navigation","Medical","Music","Music & Audio","Music & Video","News & Magazines","Parenting","Personalization","Photography","Pretend Play","Productivity","Puzzle","Racing","Role Playing","Shopping","Simulation","Social","Sports","Strategy","Tools","Travel & Local","Trivia","Video Players & Editors","Weather","Word"]
def generate_lst(genre):
    lst_temp = []
    for i in range(len(df)):
        if genre in df.Genres[i]:
            lst_temp.append(1)
        else:
            lst_temp.append(0)
    return lst_temp
d = {genre: generate_lst(genre) for genre in genres}
transfored_genres = pd.DataFrame(d)
df = pd.concat([df,transfored_genres],axis=1)

In [78]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [79]:
#unique, counts = np.unique(df.Genres.values, return_counts=True)
X = df.drop('Installs', axis=1)
y = df[['Installs']]
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [80]:
enc = OneHotEncoder(sparse=False)
category = enc.fit_transform(xTrain[['Category']])
category = pd.DataFrame(category, columns = enc.get_feature_names())
xTrain['Category'] = category

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [81]:
enc = OrdinalEncoder(categories = [['Everyone', 'Everyone 10+', 'Teen', 'Mature 17+', 'Adults only 18+','Unrated']])
content_rating = pd.DataFrame(enc.fit_transform(xTrain[['Content Rating']]),columns=['Content Rating'])
# enc = OrdinalEncoder(categories = [['Free', 'Paid']])
# Type = pd.DataFrame(enc.fit_transform(xTrain[['Type']]),columns=['Type'])
# Type

In [82]:
scaler = MinMaxScaler()
scaled_rating = scaler.fit_transform(pd.DataFrame(xTrain['Rating']))
scaled_rating = pd.DataFrame(scaled_rating,columns=['Rating'])

In [83]:
scaler = StandardScaler()
scaled_reviews = scaler.fit_transform(pd.DataFrame(xTrain['Reviews']))
scaled_reviews = pd.DataFrame(scaled_reviews,columns=['Number of Reviews'])

In [84]:
le = LabelEncoder()
transfored_target = pd.DataFrame(le.fit_transform(yTrain['Installs']),columns=['Number of Installs'])
#transfored_target

In [87]:
dropped = xTrain.drop(['Category','Genres','Content Rating','Rating','Reviews'],axis=1)
dropped = dropped.reset_index()
xTrain_concat = pd.concat([dropped, category,content_rating,scaled_rating,scaled_reviews],axis = 1)

In [88]:
#newdf.to_excel('train_set.xlsx', sheet_name='sheet1', index=False)
temp = yTrain.reset_index()
yTrain_concat = pd.concat([temp,transfored_target],axis=1)

In [89]:
xTrain_concat.to_excel('xTrain_transformed.xlsx', sheet_name='sheet1', index=False)

In [90]:
yTrain_concat.to_excel('yTrain_transformed.xlsx', sheet_name='sheet1', index=False)